# Read and manipulate some data in .pvt files

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# Here we can implement some criteria in order to modify PVT values
def calculate_new_property_value(P_value, T_value, previous_property_value):
    '''
    Here we can add some criteria in order to change values of
    a variable selected in "property_index"
    '''
    # P v = Z R T / M
    # rho = 1 / v = P M / (Z R T)
    R = 8.314 / (16.04 * 1e-3) # J/(kg.K)
    Z = 1.0
    new_property_value = P_value / (Z * R * (T_value + 273.15))
    return f"{new_property_value:.6e}"

    #if previous_property_value > 0.002:
    #    new_property_value = 0.002
    #    return f"{new_property_value:.6e}"
    #else:
    #    return f"{previous_property_value:.6e}"
    
import pathlib

# INPUT:
TABLE_NAME = "Tabela_Tampliada_200"

# Filepath
filename = pathlib.Path("W:\\" + TABLE_NAME + ".tab")

# Read original table
original_lines = [i.strip().split() for i in open(filename).readlines()]

converted_lines = original_lines.copy()

names = converted_lines[21][2][1:-1].split(',')

# Variable name
property_index = names.index("ROG")

pvt_points = [l for l in converted_lines[23:] if 'PVTTABLE' in l]
converted_pvt_points = pvt_points.copy()

some_property = []
for i, (_, _, _, point) in enumerate(pvt_points):
    line_values = point[1:-1].split(',')
    
    def get_value(index):
        return float(line_values[index])
    
    P_value = get_value(names.index("PT"))
    T_value = get_value(names.index("TM"))

    property_value = get_value(property_index)
    property_value = calculate_new_property_value(P_value, T_value, property_value)
    line_values[property_index] = property_value
    
    some_property.append(float(property_value))
    converted_point = "(" + ",".join(line_values) + ")"
    converted_pvt_points[i][3] = converted_point

converted_lines[23:] = converted_pvt_points

out_filename = pathlib.Path("W:\\" + TABLE_NAME + "_modified.tab")
converte_file = open(out_filename, "w")
with converte_file as f:
    for _list in converted_lines:
        f.write(" ".join(_list) + '\n')
        
pressures = np.array([float(i) for i in original_lines[17][2][1:-1].split(',')])
temperatures = np.array([float(i) for i in original_lines[18][2][1:-1].split(',')])
bubble_point_pressures = np.array([float(i) for i in original_lines[19][2][1:-1].split(',')])
bubble_point_temperatures = np.array([float(i) for i in original_lines[20][2][1:-1].split(',')])
some_property = np.array(some_property).reshape((temperatures.size, pressures.size))
T, P = np.meshgrid(temperatures, pressures, indexing='xy')

In [7]:
plt.figure()
plt.scatter(T, P)
plt.scatter(bubble_point_temperatures, bubble_point_pressures)
#plt.xlim(-10, 60)
#plt.ylim(-1e5, 1000e5)
plt.show()

In [11]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


fig = plt.figure()
ax = fig.gca(projection='3d')


# Plot the surface.
surf = ax.plot_surface(T, P, some_property, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_xlim(-50, 100)
#ax.set_xlim(-1.01, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()